In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import random



In [2]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

# Testing the Convergence Hypothesis

We provide an additional empirical example of partialling-out with Lasso to estimate the regression coefficient $\beta_1$ in the high-dimensional linear regression model:
  $$
  Y = \beta_1 D +  \beta_2'W + \epsilon.
  $$
  
Specifically, we are interested in how the rates  at which economies of different countries grow ($Y$) are related to the initial wealth levels in each country ($D$) controlling for country's institutional, educational, and other similar characteristics ($W$).
  
The relationship is captured by $\beta_1$, the *speed of convergence/divergence*, which measures the speed at which poor countries catch up $(\beta_1< 0)$ or fall behind $(\beta_1> 0)$ rich countries, after controlling for $W$. Our inference question here is: do poor countries grow faster than rich countries, controlling for educational and other characteristics? In other words, is the speed of convergence negative: $ \beta_1 <0?$ This is the Convergence Hypothesis predicted by the Solow Growth Model. This is a structural economic model. Under some strong assumptions, that we won't state here, the predictive exercise we are doing here can be given causal interpretation.


The outcome $Y$ is the realized annual growth rate of a country's wealth  (Gross Domestic Product per capita). The target regressor ($D$) is the initial level of the country's wealth. The target parameter $\beta_1$ is the speed of convergence, which measures the speed at which poor countries catch up with rich countries. The controls ($W$) include measures of education levels, quality of institutions, trade openness, and political stability in the country.

In [4]:
# load dataset 
data = pyreadr.read_r('C:/Users/enriq/Documents/GitHub/desktop-tutorial/3/ECO224/Labs/data/GrowthData.RData')
a = list(data.keys())
data1 = data[a[0]] 
data1

,Outcome,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,...,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
0,-0.024336,1,6.591674,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,...,0.04,0.033,0.057,0.010,47.6,17.3,0.0729,0.0667,0.348,-0.014727
1,0.100473,1,6.829794,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,...,0.64,0.173,0.274,0.067,57.1,18.0,0.0940,0.1438,0.525,0.005750
2,0.067051,1,8.895082,0.0000,0.204244,0.009186,0.260,0.325,0.201,1.00,...,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.1750,1.082,-0.010040
3,0.064089,1,7.565275,0.1997,0.248714,0.036270,0.061,0.070,0.051,1.00,...,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195
4,0.027930,1,7.162397,0.1740,0.299252,0.037367,0.017,0.027,0.007,0.82,...,2.11,0.257,0.287,0.229,34.5,17.6,0.1211,0.1308,2.500,0.003283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.031196,1,8.991064,0.0000,0.371898,0.014586,0.255,0.336,0.170,0.98,...,11.41,2.226,2.494,1.971,27.5,15.9,0.4407,0.4257,2.529,-0.011883
86,0.034096,1,8.025189,0.0050,0.296437,0.013615,0.108,0.117,0.093,1.00,...,1.95,0.510,0.694,0.362,20.2,15.7,0.1669,0.2201,25.553,-0.039080
87,0.046900,1,9.030137,0.0000,0.265778,0.008629,0.288,0.337,0.237,1.00,...,25.64,2.727,2.664,2.788,20.4,9.4,0.3238,0.3134,4.152,0.005175
88,0.039773,1,8.865312,0.0000,0.282939,0.005048,0.188,0.236,0.139,1.00,...,10.76,1.888,1.920,1.860,20.0,16.0,0.1845,0.1940,0.452,-0.029551


In [5]:
import re

In [6]:
#################################  Find Variable Names from Dataset ########################

def varlist( df = None, type_dataframe = [ 'numeric' , 'categorical' , 'string' ],  pattern = "", exclude = None ):
    varrs = []
    
    if ('numeric' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_numeric_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('categorical' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_categorical_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('string' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_string_dtype , axis = 0 ).to_list() ].tolist()
    
    grepl_result = np.array( [ re.search( pattern , variable ) is not None for variable in df.columns.to_list() ] )
    
    if exclude is None:
        result = list(compress( varrs, grepl_result ) )
    
    else:
        varrs_excluded = np.array( [var in exclude for var in varrs ] )
        and_filter = np.logical_and( ~varrs_excluded ,  grepl_result )
        result = list(compress( varrs, and_filter ) )
    
    return result   

################################# Create Variables ###############################


# Dummy Variables for Year and County Fixed Effects
#r = re.compile("X_Jfips")
#fixed = list( filter( r.match, data1.columns.to_list() ) )
#year = varlist(data1, pattern="X_Tyear")

census = []
census_var = ["^free", "^h", "^p", "^s", "^fert", "^gee","^gov", "^high", "^human", "^hyr", "^no", "^pop", "^sec", "^syr", "^tea"]

for variable in census_var:
    r = re.compile( variable )
    census = census + list( filter( r.match, data1.columns.to_list() ) )

    
################################ Variables ##################################
# Treatment Variable
d = "gdpsh465"

# Outcome Variable
y = "Outcome"

# Other Control Variables
X1 = ["bmp1l", "mort1", "mort65", "lifee065","worker65"]
X2 = ["gpop1", "invsh41", "gde1", "gvxdxe41", "pinstab1","ex1","im1","xr65","tot1"]

#################################  Partial out Fixed Effects ########################

#rdata = data1[['CountyCode']]

# Variables to be Partialled-out
varlist2 = [y] + [d] + X1 + X2 + census

rdata=data1
# Partial out Variables in varlist from year and county fixed effect
#for var_name in varlist2:
 #   form = var_name + " ~ "  + " + ".join( year ) + "+" + " + ".join( fixed )
  #  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
    

In [7]:
# load dataset

rdata_read = pyreadr.read_r("C:/Users/enriq/Documents/GitHub/desktop-tutorial/3/ECO224/Labs/data/GrowthData.RData")
data = rdata_read[ 'GrowthData' ]
n = data.shape[0]


In [8]:
# Treatment Variable
D = rdata[ f'{d}']

# Outcome Variable
Y = rdata[ f'{y}']

# Construct matrix Z
Z = rdata[ X1 + X2 + census ]

Z.shape

(90, 85)

In [9]:
data = pd.concat([Y, D, Z], axis=1)
data

,Outcome,gdpsh465,bmp1l,mort1,mort65,lifee065,worker65,gpop1,invsh41,gde1,...,secm65,secf65,secc65,seccm65,seccf65,syr65,syrm65,syrf65,teapri65,teasec65
0,-0.024336,6.591674,0.2837,0.165,0.160,3.693867,0.3469,0.0203,0.11898,0.019,...,0.75,0.17,0.13,0.21,0.04,0.033,0.057,0.010,47.6,17.3
1,0.100473,6.829794,0.6141,0.154,0.145,3.933784,0.2703,0.0185,0.12048,0.019,...,4.74,1.20,1.36,2.05,0.64,0.173,0.274,0.067,57.1,18.0
2,0.067051,8.895082,0.0000,0.027,0.024,4.273884,0.3874,0.0188,0.23098,0.040,...,33.50,39.95,15.68,13.19,18.14,2.573,2.478,2.667,26.5,20.7
3,0.064089,7.565275,0.1997,0.085,0.072,4.168214,0.3011,0.0345,0.12928,0.011,...,7.50,7.70,2.76,2.89,2.63,0.438,0.453,0.424,27.8,22.7
4,0.027930,7.162397,0.1740,0.131,0.120,3.998201,0.3314,0.0310,0.07932,0.012,...,5.37,4.78,2.17,2.23,2.11,0.257,0.287,0.229,34.5,17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.031196,8.991064,0.0000,0.017,0.011,4.309456,0.3714,0.0094,0.29184,0.034,...,47.53,42.87,10.92,10.40,11.41,2.226,2.494,1.971,27.5,15.9
86,0.034096,8.025189,0.0050,0.078,0.039,4.232656,0.4070,0.0083,0.26508,0.069,...,8.11,4.62,2.67,3.58,1.95,0.510,0.694,0.362,20.2,15.7
87,0.046900,9.030137,0.0000,0.015,0.009,4.317488,0.4808,0.0037,0.26198,0.036,...,33.26,38.04,24.09,22.48,25.64,2.727,2.664,2.788,20.4,9.4
88,0.039773,8.865312,0.0000,0.022,0.016,4.282206,0.4645,0.0026,0.20382,0.050,...,29.58,35.24,8.37,5.70,10.76,1.888,1.920,1.860,20.0,16.0


# Testing the Convergence Hypothesis

# OLS without controls

In [10]:
# OLS 
model = "Outcome ~ gdpsh465"
baseline_ols = smf.ols(model , data=data).fit()
baseline_ols_table = baseline_ols.summary2().tables[1]
print( baseline_ols_table.iloc[ 1 , 4:] )
baseline_ols_table.iloc[1, :]

[0.025   -0.010810
0.975]    0.013444
Name: gdpsh465, dtype: float64


Coef.       0.001317
Std.Err.    0.006102
t           0.215777
P>|t|       0.829661
[0.025     -0.010810
0.975]      0.013444
Name: gdpsh465, dtype: float64

The point estimate is $0.1317\%$ with the confidence interval ranging from -0.0108 to 0.0134. This
suggests that convergence anual rate is $-0.1317\%$ then the convergence hypothesis is not fulfilled .

First, we estimate the model by ols and then by an array of the modern regression methods using the double machine learning approach.

# OLS with controls

In [11]:
# define the variables
y = 'Outcome'

data_columns = list(data)
no_relev_col = ['gdpsh465', 'Outcome']

# This gives us: new_list = ['carrot' , 'lemon']
z = [col for col in data_columns if col not in no_relev_col]

In [12]:
control_formula = "Outcome" + ' ~ ' + 'gdpsh465 + ' + ' + '.join( Z.columns.to_list() )

In [13]:
control_ols = smf.ols( control_formula , data=data).fit()
control_ols_table = control_ols.summary2().tables[1]
print( control_ols_table.iloc[ 1 , 4:] )
control_ols_table.iloc[1, :]

[0.025   -0.070600
0.975]    0.051844
Name: gdpsh465, dtype: float64


Coef.      -0.009378
Std.Err.    0.029888
t          -0.313774
P>|t|       0.756019
[0.025     -0.070600
0.975]      0.051844
Name: gdpsh465, dtype: float64


Controlling for characteristics of each country, we obtain that if the convergence hypothesis is fulfilled because there is an annual convergence rate of 0.9378%

# DML algorithm

In [14]:
def DML2_for_PLM(z, d, y, dreg, yreg, nfold):
    
    # Num ob observations
    nobs = z.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
    ytil = np.zeros( len(z) ).reshape( len(z) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(z))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(z[~mask,], d[~mask,])
        yfit = yreg(z[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( z[mask,] )
        yhat = yfit.predict( z[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil), axis = 1), columns = ['ytil','dtil'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit()
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result

In [15]:
# Create main variables
Y = data['Outcome']
D = data['gdpsh465']
Z = data.drop(['Outcome', 'gdpsh465'], axis=1)


In [16]:
# as matrix
y = Y.to_numpy().reshape( len(Y) , 1 )
d = D.to_numpy().reshape( len(Y) , 1 )
z = Z.to_numpy()


# DML algorithm

# Lasso Using scikit-learn

In [17]:
def dreg(z,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, d)
    return result

def yreg(z,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, y)
    return result

DML2_lasso = DML2_for_PLM(z, d, y, dreg, yreg, 10)

0  
1  
2  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.562e-01, tolerance: 6.054e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.443e-02, tolerance: 2.177e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.01904966060499917, SE is equal to 0.010990449816948146


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.064e-01, tolerance: 6.381e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.254e-02, tolerance: 2.186e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

applying Lasso to D and Y we find that the convergence hypothesis is not fulfilled. Now the annual convergence rate is -1.9%. 

# Post - Lasso Using scikit-learn

In [18]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

In [19]:
def dreg(z,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , d )
    return result

def yreg( z , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , y )
    return result

DML2_lasso_post = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.651e-01, tolerance: 6.565e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.838e-01, tolerance: 6.565e-03
  model = cd_fast.enet_coordinate_descent(


0  
1  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.439e-02, tolerance: 1.784e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.462e-02, tolerance: 1.784e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

2  
3  
4  
5  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.965e-01, tolerance: 6.351e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.024e-01, tolerance: 6.351e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

6  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.210e-01, tolerance: 6.592e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.388e-01, tolerance: 6.592e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

7  
8  
9  
Coefficient is 0.013285717901204143, SE is equal to 0.011735321066817078


applying Post-Lasso to D and Y we find that the convergence hypothesis is not fulfilled because the annual convergence rate is -1.32%



cv.glmnet curiosities

In [20]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [21]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0  ) ).fit( z, y )
    return result

DML2_lasso_cv = DML2_for_PLM(z, d, y, dreg, yreg, 2)

C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.200e-03, tolerance: 3.509e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.646e-03, tolerance: 3.509e-03
  model =

0  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.166e-02, tolerance: 2.991e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.674e-02, tolerance: 2.991e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

1  
Coefficient is -0.03254661698549916, SE is equal to 0.012604287740127675


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.958e-05, tolerance: 1.058e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.103e-05, tolerance: 1.058e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

In [22]:
# DML with cross-validated Elastic Net:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, y )
    return result

DML2_elnet = DML2_for_PLM(z, d, y, dreg, yreg, 2 )

C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.923e-06, tolerance: 8.585e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutor

0  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.308e-05, tolerance: 1.197e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutor

1  
Coefficient is 0.0064639644503959644, SE is equal to 0.013973050348200894


If we apply both Elastic Net to estimate D and Y without taking into account the best method the minimum we obtain a coefficient of 0.646%. hence the convergence hypothesis is fulfilled.

In [23]:
#DML with cross-validated Ridge:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_ridge = DML2_for_PLM(z, d, y, dreg, yreg, 2)

C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.334e-05, tolerance: 9.108e-06
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutor

0  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.305e-05, tolerance: 1.079e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutor

1  
Coefficient is -0.02714694445563734, SE is equal to 0.01147218377663088


In [24]:
#DML with OLS:
def dreg(z,d):
    result = LinearRegression().fit( z, d )
    return result

def yreg(z,y):
    result = LinearRegression().fit( z, y )
    return result

DML2_ols = DML2_for_PLM(z, d, y, dreg, yreg, 2)

0  
1  
Coefficient is 0.014025356594863725, SE is equal to 0.0157612601586391


If we apply both OLS to estimate D and Y without taking into account the best method the minimum we obtain a coefficient of 1.4%. hence the convergence hypothesis is not fulfilled.

# Random Forest

In [25]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [26]:
#DML with Random Forest:
def dreg(z,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )
    return result

DML2_RF = DML2_for_PLM(z, d, y, dreg, yreg, 2)   # set to 2 due to computation time

C:\Users\enriq\AppData\Local\Temp/ipykernel_16092/893647714.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
C:\Users\enriq\AppData\Local\Temp/ipykernel_16092/893647714.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


0  


C:\Users\enriq\AppData\Local\Temp/ipykernel_16092/893647714.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
C:\Users\enriq\AppData\Local\Temp/ipykernel_16092/893647714.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


1  
Coefficient is -0.024160911565963932, SE is equal to 0.011469144084196707


If we apply both RF to estimate D and Y without taking into account the best method the minimum we obtain a coefficient of -0.241%. hence  the convergence hypothesis is fulfilled.

In [27]:
mods = [DML2_ols, DML2_lasso, DML2_lasso_post , DML2_lasso_cv, DML2_ridge, DML2_elnet, DML2_RF]
mods_name = ["DML2_ols", "DML2_lasso", "DML2_lasso_post" , "DML2_lasso_cv", 'DML2_ridge', 'DML2_elnet', 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_ols,DML2_lasso,DML2_lasso_post,DML2_lasso_cv,DML2_ridge,DML2_elnet,DML2_RF
RMSEY,0.008660,0.000783,0.000442,2.621360e-18,1.264421e-17,1.102513e-17,0.001114
RMSED,0.188086,0.008778,0.029387,1.875043e-16,8.881784e-16,0.000000e+00,0.032094


In [28]:
np.where(DML2_lasso_post[ 'ytil' ] == 0)[0].size   #This verfies that the function DML2_for_PLM has no errors

0


Within the four methods (ols, lasso, post lasso, random forest) the best method for D is lasso and for Y is Lasso because they give us the lowest MSE. But if we include more methods, the best method for D is lasso_cv  and for Y it is lasso_cv

In [34]:
#DML with the BEST: using lasso cv for D and Y
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0  ) ).fit( z, y )
    return result

DML2_best = DML2_for_PLM(z, d, y, dreg, yreg, 10) 

C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.434e-02, tolerance: 6.192e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.285e-03, tolerance: 6.022e-03
  model =

0  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.528e-03, tolerance: 6.313e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.895e-03, tolerance: 5.637e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

1  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.049e-03, tolerance: 5.923e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.530e-03, tolerance: 5.923e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

...

C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.549e-03, tolerance: 5.662e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.186e-03, tolerance: 5.453e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

8  


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.712e-03, tolerance: 5.646e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.728e-03, tolerance: 5.646e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

9  
Coefficient is -0.008198195070535498, SE is equal to 0.012063483729069287


C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.910e-04, tolerance: 1.998e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.881e-04, tolerance: 1.998e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\enriq\anaconda3\envs\rtutorial2\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 


We find that our estimation with the best methods tells us that if the convergence hypothesis is fulfilled since it has an annual value of 0.8%

In [35]:
table = np.zeros( ( 9 , 2 ))
table[ 0 , 0] = baseline_ols_table.iloc[ 1 , 0 ]
table[ 1 , 0] = control_ols_table.iloc[ 1 , 0 ]
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_lasso_cv['coef_est']
table[ 5 , 0] = DML2_ridge['coef_est']
table[ 6 , 0] = DML2_elnet['coef_est']
table[ 7 , 0] = DML2_RF['coef_est']
table[ 8 , 0] = DML2_best['coef_est']
table[ 0 , 1] = baseline_ols_table.iloc[ 1 , 1 ]
table[ 1 , 1] = control_ols_table.iloc[ 1 , 1 ]
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_lasso_cv['se']
table[ 5 , 1] = DML2_ridge['se']
table[ 6 , 1] = DML2_elnet['se']
table[ 7 , 1] = DML2_RF['se']
table[ 8 , 1] = DML2_best['se']

In [36]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "Least Squares with controls", "Lasso", \
             "Post-Lasso", "CV Lasso","CV Elnet", "CV Ridge", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.001,0.006
Least Squares with controls,-0.009,0.030
Lasso,0.019,0.011
Post-Lasso,0.013,0.012
CV Lasso,-0.033,0.013
CV Elnet,-0.027,0.011
CV Ridge,0.006,0.014
Random Forest,-0.024,0.011
Best,-0.008,0.012


For the selection of which method to use, we must be guided by the method that gives us the lowest MSE. In our cases, the Lasso method is the best to predict both Y and D. Subsequently, we use these methods for our debias machine learning. 
When applying DML with the best methods in our estimation we find that the convergence hypothesis is  fulfilled because the estimated coefficient has a value of -0.8%